In [ ]:
import numpy as np
import random
from tqdm import tqdm
import time
from typing import List, Tuple, Optional, Union

class SudokuDatasetGenerator:
    """
    機械学習用の数独データセットを生成するクラス。
    難易度を範囲で指定し、ランダムな数の空白を持つパズルを生成できます。
    """

    def __init__(self, grid_size: int = 9):
        if int(grid_size**0.5)**2 != grid_size:
            raise ValueError("グリッドサイズは平方数である必要があります (例: 9, 16)。")
        self.grid_size = grid_size
        self.box_size = int(grid_size**0.5)

    def _find_empty(self, board: np.ndarray) -> Optional[Tuple[int, int]]:
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if board[r, c] == 0:
                    return (r, c)
        return None

    def _is_valid(self, board: np.ndarray, num: int, pos: Tuple[int, int]) -> bool:
        row, col = pos
        if num in board[row, :]: return False
        if num in board[:, col]: return False
        box_start_row, box_start_col = row - row % self.box_size, col - col % self.box_size
        if num in board[box_start_row:box_start_row + self.box_size, box_start_col:box_start_col + self.box_size]:
            return False
        return True

    def _solve_sudoku_for_generator(self, board: np.ndarray) -> bool:
        find = self._find_empty(board)
        if not find: return True
        row, col = find
        nums = list(range(1, self.grid_size + 1))
        random.shuffle(nums)
        for num in nums:
            if self._is_valid(board, num, (row, col)):
                board[row, col] = num
                if self._solve_sudoku_for_generator(board):
                    return True
                board[row, col] = 0
        return False

    def _count_solutions(self, board: np.ndarray) -> int:
        count = 0
        def solve():
            nonlocal count
            find = self._find_empty(board)
            if not find:
                count += 1
                return
            row, col = find
            for num in range(1, self.grid_size + 1):
                if self._is_valid(board, num, (row, col)):
                    board[row, col] = num
                    solve()
                    board[row, col] = 0
                    if count > 1: return
        solve()
        return count

    def _generate_full_board(self) -> np.ndarray:
        board = np.zeros((self.grid_size, self.grid_size), dtype=int)
        self._solve_sudoku_for_generator(board)
        return board

    def _create_puzzle(self, full_board: np.ndarray, num_empty_cells: int) -> np.ndarray:
        puzzle = full_board.copy()
        cells = [(r, c) for r in range(self.grid_size) for c in range(self.grid_size)]
        random.shuffle(cells)
        emptied_cells = 0
        for row, col in cells:
            if emptied_cells >= num_empty_cells:
                break
            temp_val = puzzle[row, col]
            puzzle[row, col] = 0
            if self._count_solutions(puzzle.copy()) != 1:
                puzzle[row, col] = temp_val
            else:
                emptied_cells += 1
        return puzzle

    def generate(self, num_puzzles: int, difficulty: Union[int, Tuple[int, int]]) -> List[Tuple[np.ndarray, np.ndarray]]:
        """
        指定された数の数独データセットを生成する。

        Args:
            num_puzzles (int): 生成するパズルの数。
            difficulty (Union[int, Tuple[int, int]]):
                - int: 空白マスの数を固定。
                - Tuple[int, int]: (最小空白数, 最大空白数) の範囲でランダムに決定。
        
        Returns:
            List[Tuple[np.ndarray, np.ndarray]]: (問題, 解答) のタプルのリスト。
        """
        is_random_difficulty = isinstance(difficulty, tuple)
        if is_random_difficulty:
            min_empty, max_empty = difficulty
            print(f"{num_puzzles}個の数独パズルを生成します (空白マス: {min_empty}〜{max_empty}個)...")
            if max_empty > 55:
                 print(f"警告: 空白マスの最大数({max_empty})が非常に多いため、生成に時間がかかる可能性があります。")
        else:
            print(f"{num_puzzles}個の数独パズルを生成します (空白マス: {difficulty}個)...")
            if difficulty > 55:
                print(f"警告: 空白マスの数({difficulty})が非常に多いため、生成に時間がかかる可能性があります。")

        dataset = []
        for _ in tqdm(range(num_puzzles), desc="Generating Sudoku Puzzles"):
            if is_random_difficulty:
                num_empty_cells = random.randint(min_empty, max_empty)
            else:
                num_empty_cells = difficulty
            
            solution = self._generate_full_board()
            puzzle = self._create_puzzle(solution, num_empty_cells)
            dataset.append((puzzle, solution))
            
        return dataset


if __name__ == '__main__':
    # === データセット生成と保存の実行 ===

    # --- パラメータ設定 ---
    NUM_SAMPLES = 2100  # 生成するデータ数
    # 難易度（空白マスの数）の範囲。易しい問題から難しい問題まで含める
    # ヒント数の最小は17個（空白64個）ですが、60を超えると生成が非常に遅くなります。
    # 実用的な範囲として (30, 58) などを推奨します。
    DIFFICULTY_RANGE = (9*4, 64)
    OUTPUT_FILE = f'sudoku_dataset_{NUM_SAMPLES}.npz'

    # --- 生成開始 ---
    print("数独データセットの生成を開始します。")
    start_time = time.time()

    generator = SudokuDatasetGenerator()
    sudoku_dataset = generator.generate(
        num_puzzles=NUM_SAMPLES,
        difficulty=DIFFICULTY_RANGE
    )

    # --- NumPy配列への変換 ---
    # 問題（puzzle）と解答（solution）を別々のリストに分ける
    puzzles = [item[0] for item in sudoku_dataset]
    solutions = [item[1] for item in sudoku_dataset]

    # NumPy配列に変換 (データ型を符号なし8ビット整数にしてメモリ効率化)
    puzzles_np = np.array(puzzles, dtype=np.uint8)
    solutions_np = np.array(solutions, dtype=np.uint8)
    
    # --- NPZファイルへの保存 ---
    # savez_compressed を使うと圧縮されてファイルサイズが小さくなる
    np.savez_compressed(
        OUTPUT_FILE,
        puzzles=puzzles_np,
        solutions=solutions_np
    )

    end_time = time.time()
    
    # --- 結果の表示 ---
    print("\n--- データセット生成完了 ---")
    print(f"ファイル名: {OUTPUT_FILE}")
    print(f"保存されたデータ数: {len(puzzles_np)}")
    print(f"問題データの形状 (puzzles): {puzzles_np.shape}")
    print(f"解答データの形状 (solutions): {solutions_np.shape}")
    print(f"合計生成時間: {end_time - start_time:.2f} 秒")

    # --- 読み込みテスト ---
    print("\n--- 保存したファイルの読み込みテスト ---")
    with np.load(OUTPUT_FILE) as data:
        loaded_puzzles = data['puzzles']
        loaded_solutions = data['solutions']
        print("ファイルの読み込みに成功しました。")
        print(f"読み込んだ問題データの形状: {loaded_puzzles.shape}")
        
        # 最初の1件を表示
        print("\n最初の問題 (Puzzle):")
        print(loaded_puzzles[0])
        print("\n最初の解答 (Solution):")
        print(loaded_solutions[0])